In [1]:
import os
import sys
import types
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sas

import read_clean_quote as rcq
import calculate_trends as ct

In [2]:
n_roll    = 20
n_mom     = 20
n_rsi     = 15
n_max     = 20
days_pred = 5
predictors_df, price_df = rcq.build_quote( 'IBM', n_roll, n_mom, n_rsi, n_max, days_pred )

In [3]:
predictors_df.tail()

,Volatility,Bollinger_20,RSI_15,Momentum_20
Date,,,,
2004-02-06,0.354000,0.678560,0.448133,0.717125
2004-02-05,0.359296,0.681004,0.533333,0.718438
2004-02-04,0.364086,0.796002,0.552753,0.755391
2004-02-03,0.356890,0.815283,0.510990,0.748970
2004-02-02,0.346708,0.805872,0.505729,0.785454


In [4]:
price_df.head()

,Adj Close,Close_mean_20,Close_std_20,Diff_co,Diff_v,Close_1,Close_2,Close_3,Close_4,Close_5
Date,,,,,,,,,,
2017-03-01,181.949997,178.651213,2.990200,0.581450,0.301502,1.184512,1.421406,0.330847,0.165154,0.441625
2017-02-28,179.820007,178.211260,3.123634,0.524529,0.297068,0.234121,-0.843672,-1.007425,-0.734191,-0.244085
2017-02-27,179.399994,177.941303,3.209412,0.401208,0.374564,-1.075275,-1.238646,-0.966050,-0.477089,-0.702941
2017-02-24,181.350006,177.766759,3.220811,0.563259,0.484011,-0.165146,0.110412,0.604688,0.376381,-0.044087
2017-02-23,181.649994,177.562180,3.109244,0.480769,0.248935,0.276014,0.771108,0.542423,0.121259,-0.016512


In [8]:
# Data should be cleaned, need to plot some trends and check for correlation
# [ 'Diff_co', 'Diff_ao', 'Diff_hl', 'Diff_v'] ]
# [ 'Close_mean_n', 'Close_std_n' ]
# [ 'Momentum_n' ]
#
# Diff co, v, have different info, co same as hl basically
# Momentum 3 correlated with co, 5 a little less, 10 at limit
# HUGE autocorrelation with momentum, long momentum really tracks trends, until at/past scale of momentum

In [ ]:
from sklearn.decomposition import PCA

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm  import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [10]:
print predictors_df.shape[0]

3293


In [13]:
N_train = 2500
N_test  = 700
off     = days_pred

# Put things in right cronological order
price_df      =      price_df[::-1]
predictors_df = predictors_df[::-1]

# Labels need to be stored as a list, because will use diff CLF for each
train_label_list = []
test_label_list  = []

# Features are constant for each clf
train_features = predictors_df[:N_train]
test_features  = predictors_df[ N_train+off:N_train+off+N_test]

# Loop over offsets so have training data for each day
for offset in range( 1, off+1 ):
    
    # Train data from 0 to N_train
    # Train labels from ^+offset
    # Test data from N_train+o to N_train + N_test + o
    # Test labels from ^+offset
    
    train_label_list.append(  price_df.ix[:N_train                       ,'Close_'+str(offset)].values )
    test_label_list.append(   price_df.ix[ N_train+off:N_train+off+N_test,'Close_'+str(offset)].values )

In [32]:
my_pca = PCA( n_components=3 )
my_pca.fit( train_features )
trans_train_features = my_pca.transform( train_features )
trans_test_features  = my_pca.transform( test_features  )
print my_pca.explained_variance_ratio_

[ 0.77235332  0.10325975  0.08072967]


In [39]:
clf_list  = []
pred_list = []

for i in range( 0, off ):
    
    clf_list.append( SVR( C=100, epsilon=0.02 ) )
    
    clf_list[i].fit( trans_train_features, train_label_list[i] )
    
    pred = clf_list[i].predict( trans_train_features )
    
    pred_list.append( pred )

In [40]:
# i price, p1, p2, p3, p4, p5
# ' '
for i in range( 100, 110 ):
    
    print i, price_df.ix[i,'Adj Close'], '     ',
    
    if ( i < 105 ):
        for j in range( 0, off ):
            print '%10.5f' % ((pred_list[j][i]/100. + 1. ) * price_df.ix[i,'Adj Close']), '  ',
        
    print ' '
        
#print clf_list[i].score( train_features, (train_label_list[i]/100.+1.0)*price_df.ix[0,'Adj Close'] ), price_df.ix[0,'Adj Close'],
#print clf_list[i].predict( test_features.iloc[0] ), test_label_list[i][0]

100 69.681656         69.59013      69.54204      69.47028      69.40700      69.43264     
101 69.028022         68.87176      68.71511      68.56464      68.38897      68.26445     
102 68.701208         68.49772      68.29288      68.11568      67.90709      67.72055     
103 68.592271         68.37693      68.16985      68.00101      67.79722      67.61639     
104 68.086484         67.65584      67.28219      67.03985      66.81402      66.52722     
105 67.728544        
106 66.685845        
107 66.413501        
108 65.09068        
109 65.27743        
